# **TFM Izar Ortin Riquelme**
## “Máster Big Data y Business Analytics UNED 2019/2020 - UNED”

# i. Abstract

Este trabajo analiza la efectividad comparativa de los dos principales modelos utilizados en problemas de tipo NLP, NLI y NLU, que son BERT y XML-RoBERTa, así como una propuesta de actuación a la hora de abordar este tipo de problemas; incorporando una mayor cantidad de datos similares con los que mejorar la capacidad predictiva de los mismos, realizar data augmentation y, finalmente, realizar un fine tuning sobre los hiperparámetros del modelo.

Realizamos todo el entrenamiento del modelo en TPUs desarrollados por Google, y estructuramos todas las partes del modelo a través de funciones de la biblioteca Keras.

Finalmente, al tratarse de una competición de kaggle, realizaremos un submission y trataremos de alcanzar la máxima puntuación posible.

# **ii.	Introducción y objetivos buscados**

Este trabajo es parte de la evaluación del “Máster Big Data y Business Analytics UNED 2019/2020 - UNED”, y en él se desarrolla el análisis, dentro de la disciplina deep learning, de la competición de Kaggle “Contradictory, My Dear Watson. Detecting contradiction and entailment in multilingual text using TPUs”.

El **objetivo general** de este trabajo es encontrar un modelo NLI (Natural Language Inference) que asigne correctamente etiquetas correspondientes a implicación, neutralidad o contradicción a pares de premisas e hipótesis, sea cual sea el idioma en el que se encuentren y mejorar su capacidad de predicción.

Los **objetivos específicos**, son:

1.	Encontrar el modelo de lenguaje que mejor clasifique los pares de sentencias del data-set a analizar, entre BERT y XML-Roberta.


2.	Entrenar el modelo escogido con bases de datos distintas a las proporcionadas inicialmente.


3.	Realizar un "data augmentation", para evitar problemas de sobreentrenamiento y mantener la estructura del data set original.


# iii. Data preparation y carga de librerias necesarias

Lo primero que haremos es actualizar la librería transformers de Hugging Face, para evitar problemas a la hora de conectar con ciertos tokenizadores, como ocurría en versiones anteriores.

In [ ]:
!pip install transformers==3.0.2

Establecemos las rutas de carga de archivos a los input del notebook.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
os.environ["WANDB_API_KEY"] = "0" ## Para silenciar el aviso que nos daría Colab al no darle una KEY válida.

Importamos el resto de librerías que utilizaremos:

In [ ]:
#Las relacionadas con transformers de Hugging Face.#Las relacionadas con transformers de Hugging Face.

import transformers 
from transformers import BertTokenizer, TFBertModel, AutoTokenizer, TFAutoModel

#Las relacionadas con TensorFlow

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam, SGD, Adamax
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from kaggle_datasets import KaggleDatasets

#El train-test split de Scikit-learn.

import sklearn
from sklearn.model_selection import train_test_split

#Las herramientas gráficas necesarias.

import plotly.express as px
import matplotlib.pyplot as plt

#Para acceder a las librerías de datasets de Hugging Face.

!pip install nlp
import nlp

#Resto de funciones

import datetime
from tqdm.notebook import tqdm
import random
from PIL import Image
import requests
from io import BytesIO
import io
import PIL

Comprobamos que tenemos la versión de transformers correcta.

In [ ]:
transformers.__version__

In [ ]:
Image.open("../input/tf-logo/tf_logo.png")

## Configuración de TensorFlow

TensorFlow es una plataforma de código abierto de extremo a extremo para el aprendizaje automático. Cuenta con un ecosistema integral y flexible de herramientas, bibliotecas y recursos de la comunidad que les permite a los investigadores impulsar un aprendizaje automático innovador y, a los desarrolladores, compilar e implementar con facilidad aplicaciones con tecnología de AA.

Configuraremos nuestro TPU, siendo este uno de tipo V3-8.

Este tipo define tanto la versión de TPU (v3) como la cantidad de núcleos de "TPU-v3" de que dispone.

Contamos con 8 y la cantidad de memoria de TPU que está disponible para la carga de trabajo de aprendizaje automático; que son 128 GiB, así como las zonas disponibles, que en este caso son us-central1-a,b y f.

In [ ]:
Image.open("../input/tpu-v38/tpu--sys-arch4 1.png")

In [ ]:
#Detectaremos el hardware requerido, y nos devolverá la distribución adecuada.

try:
    #No es necesario introducir un valor en el "resolver", ya que la enviroment variable TPU_NAME ya está establecida en Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    
    #Le brindamos dicha información al tf.config.experimental_connect_to_cluster, que conecta con el cluster dado    
    tf.config.experimental_connect_to_cluster(tpu)
    
    #Inicializamos el TPU.
    tf.tpu.experimental.initialize_tpu_system(tpu)
    
    #Y definimos la estrategia de distribución.
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    
except ValueError:
    strategy = tf.distribute.get_strategy() 
    print('Number of replicas:', strategy.num_replicas_in_sync)

## Descargamos dataset y EDA

El set de entrenamiento contiene una premisa, una hipótesis, un nivel dado (0= Implicación, 1=neutral y 2=contradicción) y el lenguaje del texto. 

Son 12120 binomios de hipótesis/premisa en varios idiomas (Árabe, búlgaro, chino, alemán, griego, inglés, español, francés, hindi, ruso, swahili, tailandés, turco, urdu y vietnamita). 

Puede comprobarse en: https://www.kaggle.com/c/contradictory-my-dear-watson/data

In [ ]:
train = pd.read_csv("../input/contradictory-my-dear-watson/train.csv")

test = pd.read_csv("../input/contradictory-my-dear-watson/test.csv")

submission = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/sample_submission.csv')

Revisamos las primeras filas. Como puede observarse, se respetan tanto las tildes como los distintos grafos utilizados por las diferentes lenguas.

### Train, Validation y Test Sets en Machine Learning

A continuación, explicaremos las diferencias entre estos tres conceptos.

1. Training Dataset

**Definición:** Una muestra de datos utilizada para ajustar el modelo.

En caso de una red neuronal, los weights y bias. El modelo observa y aprende de estos datos.

2. Validation Dataset

**Definición:** Es una muestra de datos utilizada para proporcionar una evaluación del entrenamiento del modelo, **mientras** se ajustan los hiperparametros en el entrenamiento. La evaluación se va volviendo más sesgada a medida que el conjunto de datos de validación se va incorporando al modelo en las distintas iteraciones.

Se usan los datos para ajustar los hiperparámetros. Así, el modelo ve los datos, pero no aprende de ellos. El modelo recoge los resultados evaluados en el conjunto de validación y actualiza los hiperparámetros. 

Se entiende así que el conjunto de validación afecta al modelo, pero de manera indirecta, es decir: a través de los resultados obtenidos al evaluar el modelo con dichos datos.

También se le conoce (aunque con menos frecuencia) como Development Dataset, y tiene sentido ya que ayuda al modelo en su etapa de "desarrollo".

3. Test Dataset

**Definición:** Es el conjunto de datos utilizado para evaluar de manera **insesgada** el ajuste final del modelo, que ha sido ajustado con los datos de entrenamiento. Este conjunto de datos solo se utiliza cuando el modelo esta completamente entrenado, y dicho conjunto debe estar cuidadosamente muestreado con las diferentes clases al que se enfrentaría el modelo en el mundo real.

Las diferencias entre el concepto de test en data science y kaggle, las veremos al final de este trabajo.

Un esquema aproximado de las proporciones de datos que debe incorporar cada conjunto de datos, sería el siguiente:

In [ ]:
response = requests.get('https://miro.medium.com/max/700/1*Nv2NNALuokZEcV6hYEHdGA.png')
image_bytes = io.BytesIO(response.content)

img = PIL.Image.open(image_bytes)
img

### EDA

In [ ]:
train.head()

Revisamos un binomio de frases, empezando por la premisa

In [ ]:
train.premise.values[1]

Ahora la hipótesis.

In [ ]:
train.hypothesis.values[1]

**Y su label, que como se puede observar, corresponde a una contradicción.

In [ ]:
train.label.values[1]

Revisamos la distribución de idiomas en el training set.

In [ ]:
labels, frequencies = np.unique(train.language.values, return_counts = True)

plt.figure(figsize = (10,10))
plt.pie(frequencies,labels = labels, autopct = '%1.1f%%')
plt.show()

Veamos cuantos valores son nulos:

In [ ]:
train.isnull().sum()

Como podemos observar, no hay ningún valor null

# iv. Evaluación de modelos

## Definición de las principales variables 

In [ ]:
#Definimos los dos modelos a estudiar
model_xml_roberta = 'jplu/tf-xlm-roberta-large'

model_bert = 'bert-base-multilingual-cased'

#Definimos el número de epochs a iterar y el max_len
n_epochs = 10
max_len = 80

# El tamaño de nuestro batch_size dependerá del número de réplicas en nuestra estrategia
batch_size = 16 * strategy.num_replicas_in_sync

## Características de los modelos

### Overview

* BERT fue lanzado en noviembre de 2018 por Google, con modelos para Inglés y Chino, poco después incluyeron un modelo más, el mBERT, que incluía 104 lenguas diferentes.

* XLM-RoBERTa fue lanzando en noviembre de 2019 por Facebook, incluyendo 100 lenguas diferentes.

Si bien es cierto que ambos son modelos multilenguaje, hay un elemento que en un principio parece darle un mayor protagonismo a XLM-R sobre BERT:

Analistas de la Charles University en Praga, encontraron que mBERT internamente clusteriza lenguas en "familias" y representa oraciones similares de manera diferente en diferentes idiomas. Esto ayudaría a explicar que clasifique mejor en unas pocas lenguas (ingles sobre todo) frente a otras.
Esta diferencia, tienen un efecto en la precisión a la hora de clasificar y pueden observarse en la siguiente tabla (obtenida del  Unsupervised Cross-lingual Representation Learning at Scale by Alexis Conneau, Kartikay Khandelwal, Naman Goyal, Vishrav Chaudhary, Guillaume Wenzek, Francisco Guzmán, Edouard Grave, Myle Ott, Luke Zettlemoyer and Veselin Stoyanov). 


In [ ]:
Image.open("../input/table-accurancy-models/Table accurancy models.png")

### Especificaciones y funcionamiento de BERT

* **Arquitectura del modelo**

Para empezar, hemos de saber que existen dos tamaños distintos para el modelo BERT:

1. BERT BASE: con 12 capas, 12 attention heads y 110 millones de parámetros.

2. BERT LARGE: con 24 capas, 16 attention heads y 340 millones de parámetros.

In [ ]:
response = requests.get('http://jalammar.github.io/images/bert-base-bert-large-encoders.png')
image_bytes = io.BytesIO(response.content)

img = PIL.Image.open(image_bytes)
img

Bert es básicamente un Transformer Encoder stack entrenado, en el que se le introducen unos inputs compuestos de tokens, los cuales son procesados y arroja unos outputs numericos.

El esquema de entrada seria así:

In [ ]:
response = requests.get('http://jalammar.github.io/images/bert-encoders-input.png')
image_bytes = io.BytesIO(response.content)

img = PIL.Image.open(image_bytes)
img

La estructura de estos tokens, son explicados más adelante en el apartado "Similitudes".

En cuanto al output, sería un vector numérico, siguiendo el siguiente esquema:

In [ ]:
response = requests.get('http://jalammar.github.io/images/bert-output-vector.png')
image_bytes = io.BytesIO(response.content)

img = PIL.Image.open(image_bytes)
img

* **Pre-Procesamiento de texto**

Los desarrolladores de BERT, han establecido una serie de reglas a la hora de introducir los imputs.

Cada input embedding es una combinación de 3 embeddings:

1. Position Embeddings: El modelo usa y conoce la posición de los distintos vocablos en el input introducido, lo cual lo diferencia de las Redes Neuronales Recurrentes (RNN) que no son capaces de captar información de "orden" o "secuencia".

2. Segment Embeddings: Aprende una inserción única para la primera y segunda oración y ayuda al modelo a distinguir entre ellas. Esto también genera problemas a la hora de calcular loss functions si una misma primera oración se repite con distintas segundas oraciones, lo cual ocurre en la base de datos SNLI.

3. Token Embeddings: De esta última es de donde aprenden los distintos token específicos procedentes del WordPiece.

In [ ]:
response = requests.get('https://cdn.analyticsvidhya.com/wp-content/uploads/2019/09/bert_emnedding.png')
image_bytes = io.BytesIO(response.content)

img = PIL.Image.open(image_bytes)
img

* **Masked Language Modeling**

BERT es un modelo altamente bidireccional.

A diferencia de modelos contexto de izquierda a derecha (o viceversa), que son entrenados para predecir la siguiente palabra de la oración, y por lo tanto susceptibles a error por perdida de información, como ocurre con el modelo GPT; el modelo ELMo, trató de solventar dicho problema entrenando dos LSTM de izquierda a derecha y de derecha a izquierda y concatenandolos a posteriori, pero no resultó lo suficientemente preciso.

A continuación podemos ver el esquema de flujos de informacion de BERT, OpenAI-GPT y ELMo:

In [ ]:
response = requests.get('https://cdn.analyticsvidhya.com/wp-content/uploads/2019/09/bert-vs-openai-.jpg')
image_bytes = io.BytesIO(response.content)

img = PIL.Image.open(image_bytes)
img

Las flechas representan la dirección de la información de una capa a otra. Se puede apreciar facilmente que BERT es bidireccional, GPT es unidireccional y ELMo es artificialmente "bidireccional".



### Especificaciones y funcionamiento de XML-RoBERTa

* **Arquitectura del modelo**

Para empezar, hemos de saber que existen dos tamaños distintos para el modelo BERT:

XML-R BASE: con 12 capas, 12 attention heads y 270 millones de parametros.

XML-R LARGE: con 24 capas, 16 attention heads y 550 millones de parametros.

* **Training Data**

La base de datos usada para entrenar a XML-RoBERTa, es significativamente mayor que la usada por BERT, ya que este usa Wiki-100, una base de datos procedente de Wikipedia, mientras que XML-R usa CommonCrawl. Se utilizó CommonCrawl ya que aumenta considerablemente el conjunto de datos, sobre todo para idiomas de bajos recursos, como el Birmano y el Suajili. Comprobamos esto en el gráfico mostrado a continuación:

In [ ]:
response = requests.get('https://miro.medium.com/max/2272/1*9cRchmIyxP4LUnONXLM82g.png')
image_bytes = io.BytesIO(response.content)

img = PIL.Image.open(image_bytes)
img

* **Tokenización multilenguaje**

Se entrena el Sentence Piece Model (SPM) y se aplica directamente en los datos de texto sin procesar. No se observa pérdida de rendimiento con respecto a modelos entrenados con preprocesamiento específico del lenguaje y codificación tipo byte-pair, como puede observarse en la siguiente figura.

In [ ]:
response = requests.get('https://images.deepai.org/converted-papers/1911.02116/x7.png')
image_bytes = io.BytesIO(response.content)

img = PIL.Image.open(image_bytes)
img


### Similitudes

Ambos modelos tienen unos inputs parecidos.

Se necesita codificar todos nuestros pares de premisas/hipótesis y prepararemos los inputs necesarios para el modelo: input word IDs, input masks y  input type IDs; siendo:

•	*input word IDs*: Las palabras (o segmentos de palabras) convertidos a IDs.

•	*Input masks*: Los IDs que ayudan al modelo a distinguir entre los tokens relevantes de los de relleno o padding, que son añadidos para que ambas sentencias tengan una misma longitud.

•	 *input type IDs*: Se le asignan ceros tanto al CLS como a la primera sentencia y unos a la segunda.

La principal diferencia en este punto es la configuración de los datos de entrada en los *input word IDS*:

•	BERT: Son del tipo [CLS] A [SEP] B [SEP], siendo A y B un par de sentencias.

•	XML-Roberta: Son del tipo < s > A < /s > < /s > B < /s >, siendo A y B un par de sentencias.

## BERT

In [ ]:
# Cargamos el tokenizador correspondiente
tokenizer_bert = AutoTokenizer.from_pretrained(model_bert)

In [ ]:
# Transformaremos el texto en listas, para poder introducirlas en el batch_encode_plus
train_text_bert = train[['premise', 'hypothesis']].values.tolist()
test_text_bert = test[['premise', 'hypothesis']].values.tolist()

# Ahora utilizaremos el tokenizador que hemos preparado previamente.
train_encoded_bert = tokenizer_bert.batch_encode_plus(
    train_text_bert,
    pad_to_max_length=True,
    max_length=max_len
)

test_encoded_bert = tokenizer_bert.batch_encode_plus(
    test_text_bert,
    pad_to_max_length=True,
    max_length=max_len
)

Cortamos el conjunto de entrenamiento original en entrenamiento y validación

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(
    train_encoded_bert['input_ids'], train.label.values, 
    test_size=0.2, random_state=2020
)

x_test = test_encoded_bert['input_ids']

Convertimos los diferentes conjuntos de datos en **tf.data.Dataset.**

A la hora de operar con TPUs, antes que suministrar diccionarios al modelo, es mucho más interesante mover la información a través de tf.data.Dataset, ya que se canaliza de una manera mucho más eficiente y ágil.

Lo que entre otras ventajas, recorta sensiblemente los tiempos que necesita el modelo para realizar las distintas iteraciones de la estimación (epochs).


In [ ]:
auto = tf.data.experimental.AUTOTUNE

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(batch_size)
    .prefetch(auto)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(batch_size)
    .cache()
    .prefetch(auto)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(batch_size)
)

Ahora construiremos el modelo dentro del TPU, lo haremos a través de la orden strategy.scope() de forma que todo estará dentro del scope de la strategy y no lo correrá el CPU por defecto.

In [ ]:
with strategy.scope():
    # Primero cargamos la capa de codificador.
    transformer_encoder = TFAutoModel.from_pretrained(model_bert)

    # Definimos los inputs tokenizados
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")

    # Ahora, codificamos los inputs segun el encoder que hemos definido anteriormente.
    sequence_output = transformer_encoder(input_ids)[0]

    # Extraemos los tokens utilizados para clasificar, en este caso [CLS]
    cls_token = sequence_output[:, 0, :]

    # La última capa es la que pasamos a través de softmax para la aplicación del uso de probabilidades. Con 3 niveles, ya que son 3 posibles resultados (0,1 y 2)
    out = Dense(3, activation='softmax')(cls_token)

    # Construimos y compilamos el modelo.
    model = Model(inputs=input_ids, outputs=out)
    model.compile(
        Adam(lr=1e-5), 
        loss='sparse_categorical_crossentropy', 
        metrics=['accuracy']
    )

model.summary()

Y finalmente lo entrenaremos:

In [ ]:
n_steps = len(x_train) // batch_size

train_history_bert = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=n_epochs
)

Ahora revisaremos el comportamiento de las funciones de pérdida y de precisión, para poder resolver si el modelo esta sobreentrenado o hay que seguir iterando.

In [ ]:
# list all data in history
print(train_history_bert.history.keys())
# summarize history for loss
plt.plot(train_history_bert.history['loss'])
plt.plot(train_history_bert.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for accuracy
plt.plot(train_history_bert.history['accuracy'])
plt.plot(train_history_bert.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## XML-Roberta

In [ ]:
# Cargamos el tokenizador correspondiente
tokenizer_xml_roberta = AutoTokenizer.from_pretrained(model_xml_roberta)

In [ ]:
# Transformaremos el texto en listas, para poder introducirlas en el batch_encode_plus
train_text_xml_roberta = train[['premise', 'hypothesis']].values.tolist()
test_text_xml_roberta = test[['premise', 'hypothesis']].values.tolist()

# Ahora utilizaremos el tokenizador que hemos preparado previamente.
train_encoded_xml_roberta = tokenizer_xml_roberta.batch_encode_plus(
    train_text_xml_roberta,
    pad_to_max_length=True,
    max_length=max_len
)

test_encoded_xml_roberta = tokenizer_xml_roberta.batch_encode_plus(
    test_text_xml_roberta,
    pad_to_max_length=True,
    max_length=max_len
)

Cortamos el conjunto de entrenamiento original en entrenamiento y validación.

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(
    train_encoded_xml_roberta['input_ids'], train.label.values, 
    test_size=0.2, random_state=2020
)

x_test = test_encoded_xml_roberta['input_ids']

Convertimos los diferentes conjuntos de datos en tf.data.Dataset.

In [ ]:
auto = tf.data.experimental.AUTOTUNE

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(batch_size)
    .prefetch(auto)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(batch_size)
    .cache()
    .prefetch(auto)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(batch_size)
)

Ahora construiremos el modelo dentro del TPU, lo haremos a través de la orden strategy.scope() de forma que todo estará dentro del scope de la strategy y no lo correrá el CPU por defecto.

In [ ]:
with strategy.scope():
    # Primero cargamos la capa de codificador.
    transformer_encoder = TFAutoModel.from_pretrained(model_xml_roberta)

    # Definimos los inputs tokenizados
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")

    # Ahora, codificamos los inputs segun el encoder que hemos definido anteriormente.
    sequence_output = transformer_encoder(input_ids)[0]

    # Extraemos los tokens utilizados para clasificar, en este caso <s>
    cls_token = sequence_output[:, 0, :]

    # La última capa es la que pasamos a través de softmax para la aplicación del uso de probabilidades. Con 3 niveles, ya que son 3 posibles resultados (0,1 y 2)
    out = Dense(3, activation='softmax')(cls_token)

    # Construimos y compilamos el modelo.
    model = Model(inputs=input_ids, outputs=out)
    model.compile(
        Adam(lr=1e-5), 
        loss='sparse_categorical_crossentropy', 
        metrics=['accuracy']
    )

model.summary()

Y finalmente lo entrenaremos:

In [ ]:
n_steps = len(x_train) // batch_size

train_history_xml_roberta = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=n_epochs
)

Ahora revisaremos el comportamiento de las funciones de pérdida y de precisión, para poder resolver si el modelo esta sobreentrenado o hay que seguir iterando

In [ ]:
# list all data in history
print(train_history_xml_roberta.history.keys())
# summarize history for loss
plt.plot(train_history_xml_roberta.history['loss'])
plt.plot(train_history_xml_roberta.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for accuracy
plt.plot(train_history_xml_roberta.history['accuracy'])
plt.plot(train_history_xml_roberta.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

### Overfitting y Undercomputing

El problema central en machine learning es el aprendizaje supervisado. 

Los algoritmos de aprendizaje esencialmente buscan un espacio de funciones (llamadas normalmente hypothesis class) para una función dada que encaje en un conjunto de datos dado.

Esto lo diferencia de la programación tradicional a un nivel primario, así, mientras que en la programación tradicional se trata de escoger un programa o función dada, introducirle datos, y nos da un output (o respuesta esperada), en machine learning, se trata de lo opuesto; se le proporciona datos y respuestas y nos tiene que devolver un programa o función que encaje a ambos.

In [ ]:
response = requests.get('https://miro.medium.com/max/398/1*BfvKeP4ykqi4J4C5g4EZzg.png')
image_bytes = io.BytesIO(response.content)

img = PIL.Image.open(image_bytes)
img

Debido a que el número de hypothesis functions es exponencialmente enorme, no se puede examinar individualmente cada una de ellas. En lugar de eso, se suele formalizar definiendo una funcion objetiva (por ejemplo, el número de puntos incorrectamente predichos) y aplicando varios algoritmos que minimicen esa función objetivo (suelen ser algoritmos heurísticos como el *gradient descent*).

La clave del problema surge de la tarea asignada al proble de machine-learning. Un algoritmo es entrenado con un conjunto de datos de entrenamiento, pero una vez entrenado es aplicado para realizar predicciones de datos nuevos. El **objetivo último** es maximizar su precisión predictiva de estos nuevos datos, no la precisión en los datos de entrenamiento.

Si trabaja excesivamente en estos datos de entrenamiento, absorbe sus peculiaridades (que no tienen por qué repetirse en los nuevos datos que queremos tratar), por lo tanto, genera un ruido no deseado en lugar de encontrar una regla de predicción general. Este fenómeno se denomina **overfitting** o sobre-entrenamiento.

Así, un algoritmo demasiado codicioso a la hora de encontrar un árbol de decisión o red neuronal que minimice la función objetivo al máximo, en teoría, la óptima, podría no ser válido para nuestro objetivo último, mientras que otro que encuentre un resultado sub-óptimo durante el entrenamiento, podría resultarnos el resultado óptimo a la hora de predecir nuevos datos. A este concepto se le denomina *undercomputing*, y actuaría efectivamente a la hora de evitar el overfitting.

### Conclusiones analisis de modelos

Aunque ambos modelos sobreentrenan a partir de la 3ª iteración, (momento en el que se cruzan las funciones de pérdida del entrenamiento y validación) en dicho punto la precisión del XLM-Roberta es mayor que la de BERT, por lo que utilizaremos el **XML-Roberta** a partir de este punto.

# v. Dataset Extra

A la hora de usar datasets extra hay que tener en cuenta que si el conjunto de datos es diferente, probablemente sus características tengan diferentes significados y tamaños.

Estos datos deberían reunir el significado, la interpretación y el tamaño del conjunto de entrada original a estudiar, ya que de otra forma, la interpretabilidad y la confianza del modelo serían menos útiles que el conjunto original de datos únicamente.

Por lo que en nuestro caso, lo hacemos con un gran cuidado y fijando unos parámetros:

1. Tamaño: Las frases introducidas en todos los conjuntos que añadimos al modelo, son de una longitud similar.

2. Interpretación: En este punto también coinciden, ya que son todas pares de sentencias de premisas e hipótesis junto con un label de relación entre ambas.

3. Estructura de los datos: Si bien XNLI tiene una distribución de idiomas parecida al conjunto original, no es el mismo, y una vez realicemos el data augmentation, esta proporción quedaría más desvalanceada. De esta forma,  solucionamos esta eventualidad introduciendo una muestra del MNLI (que sólo son sentencias en inglés) para mantener la proporción de idiomas del conjunto de datos original.

Ahora cargaremos dos conjuntos de datos extra;

El corpus **MNLI**: Es una colección de múltiples fuentes con 433.000 pares de oraciones anotadas con información sobre la vinculación semántica entre ellas. Todas en inglés.

El corpus **XNLI**: Es una colección de 7500 pares de de oraciones, traducidas a 15 idiomas. Lo que da un total de 112.500 pares de oraciones.

### MNLI

Cargamos el set.

In [ ]:
mnli = nlp.load_dataset(path='glue', name='mnli')

Convertimos a dataframe los subconjuntos, ya que en este dataset hay 3 subconjuntos de datos "train", "validation_matched" y "validation_mismatched"

In [ ]:
mnli_train_df = pd.DataFrame(mnli['train'])

mnli_train_df = mnli_train_df[['premise', 'hypothesis', 'label']]

mnli_train_df['lang_abv'] = 'en'

Esta es una base de datos que no nos concuerda con la proporción de idiomas utilizados en nuestro data set original, por lo que sólo utilizaremos las filas necesarias para ajustar el dataset final a la proporción de idiomas hallada en el dataset original.

In [ ]:
mnli_train_df.head(10)

In [ ]:
mnli_sample= mnli_train_df.sample(n = 40000,random_state= 2020)

In [ ]:
mnli_sample.head(10)

### XNLI

Cargamos el dataset

In [ ]:
xnli = nlp.load_dataset(path='xnli')

Convertimos a dataframe. En éste, en cambio, sólo hay un subconjunto de datos

In [ ]:
buffer = {
    'premise': [],
    'hypothesis': [],
    'label': [],
    'lang_abv': []
}


for x in xnli['validation']:
    label = x['label']
    for idx, lang in enumerate(x['hypothesis']['language']):
        hypothesis = x['hypothesis']['translation'][idx]
        premise = x['premise'][lang]
        buffer['premise'].append(premise)
        buffer['hypothesis'].append(hypothesis)
        buffer['label'].append(label)
        buffer['lang_abv'].append(lang)
        
# convert to a dataframe and view
xnli_valid_df = pd.DataFrame(buffer)
xnli_valid_df = xnli_valid_df[['premise', 'hypothesis', 'label', 'lang_abv']]

Revisamos el subconjunto

In [ ]:
xnli_valid_df.head(10)

# vi. Data Augmentation

El proceso de Data Augmentation puede actuar como un regularizador en la prevención de overfitting en redes neuronales y mejorar el rendimiento en casos de problemas desbalanceados (como es nuestro caso). 

Pero esto tiene una serie de limitaciones; al realizar cualquier proceso de Data Augmentation, corremos el riesgo de modificar el significado de las variables de estudio de tal forma que ya no correspondan a los labels asignados, por lo que debe de hacerse con extremo cuidado y siendo conocedores de cómo afecta cada cambio introducido en el modelo a entrenar.

Si bien es posible realizar este proceso en *data-space* (aumentando el numero de datos pero manteniendo las características) o hacerlo en *feature-space* (modificando ciertas caracteristicas creando nuevos datos con estas nuevas características), es más recomendable realizarlo en data-space, como indican diferentes estudios, entre ellos "Understanding data augmentation for classification: when to warp?" de Sebastien C. Wong, Adam Gatt,Victor Stamatescu and Mark D. McDonnell, en orden de mantener la adecuada correlación de los labels.

Un ejemplo de esto, sería rotar o deformar una imagen o variar la pose del objeto en un caso de clasificación de imágenes:

* Mientras que rotarlo, deformarlo o transformarlo a blanco y negro sería un cambio en **data-space**:

In [ ]:
response = requests.get('https://miro.medium.com/max/750/0*cZwrV8EyfJSeunag.png')
image_bytes = io.BytesIO(response.content)

img = PIL.Image.open(image_bytes)
img

Realizar un cambio en la pose del objeto, sería un cambio en **feature-space**:

In [ ]:
response = requests.get('https://d3i71xaburhd42.cloudfront.net/f3ee8dcaaad5f47f347354fe5d740096097cbed5/1-Figure1-1.png')
image_bytes = io.BytesIO(response.content)

img = PIL.Image.open(image_bytes)
img

En orden de aumentar el ponderamiento del conjunto de datos original, y que por lo tanto, tenga más peso sobre el análisis, vamos a realizar un data augmentation de tipo data-space:

1. Traduciendo las frases no inglesas a inglés.

2. Traduciendo frases en inglés a lenguas escogidas aleatoriamente.

### Descargaremos las librerías necesarias

In [ ]:
!pip -q install googletrans
import gc
from googletrans import Translator
from dask import bag, diagnostics
import seaborn as sns

Estableceremos una semilla, para mantener la elección de lenguas.

In [ ]:
SEED = 42
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)

Definimos las funciones necesarias para traducir el conjunto de datos.

In [ ]:
def translate(words, dest):
    dest_choices = ['zh-cn',
                    'ar',
                    'fr',
                    'sw',
                    'ur',
                    'vi',
                    'ru',
                    'hi',
                    'el',
                    'th',
                    'es',
                    'de',
                    'tr',
                    'bg'
                    ]
    if not dest:
        dest = np.random.choice(dest_choices)
        
    translator = Translator()
    decoded = translator.translate(words, dest=dest).text
    return decoded

def trans_parallel(df, dest):
    premise_bag = bag.from_sequence(df.premise.tolist()).map(translate, dest)
    hypo_bag =  bag.from_sequence(df.hypothesis.tolist()).map(translate, dest)
    with diagnostics.ProgressBar():
        premises = premise_bag.compute()
        hypos = hypo_bag.compute()
    df[['premise', 'hypothesis']] = list(zip(premises, hypos))
    return df

Finalmente, realizamos la traducción:

In [ ]:
eng = train.loc[train.lang_abv == "en"].copy().pipe(trans_parallel, dest=None)
non_eng =  train.loc[train.lang_abv != "en"].copy().pipe(trans_parallel, dest='en')
train_data_translate = train.append([eng, non_eng])

# vii. Entrenamiento final del modelo

### Análisis del conjunto final 
En esta parte final del trabajo, unificaremos todas los dataset, y entrenaremos al modelo con ellos.

In [ ]:
frames = [train_data_translate, xnli_valid_df, mnli_sample]
train_data_def = pd.concat(frames)

Y reiniciearemos el TPU, ya que se encuentra saturado por los cálculos anteriores y no sería capaz de mover el modelo final

In [ ]:
tf.tpu.experimental.initialize_tpu_system(tpu)

Comprobaremos que mantienen tanto el formato, como la proporción de lenguas inglesas y no inglesas de comienzo.

In [ ]:
train_data_def.head()

In [ ]:
labels, frequencies = np.unique(train_data_def.lang_abv.values, return_counts = True)

plt.figure(figsize = (10,10))
plt.pie(frequencies,labels = labels, autopct = '%1.1f%%')
plt.show()

Podemos observar que el conjunto de datos final, guarda unas proporciones de idiomas muy similar al conjunto original, así como al conjunto de test de esta competición.

Lo cual es muy interesante de cara al último paso: la confección del submission.

### Entrenamiento del modelo definitivo

In [ ]:
# Transformaremos el texto en listas, para poder introducirlas en el batch_encode_plus
train_text_xml_roberta = train_data_def[['premise', 'hypothesis']].values.tolist()
test_text_xml_roberta = test[['premise', 'hypothesis']].values.tolist()

# Ahora utilizaremos el tokenizador que hemos preparado previamente.
train_encoded_xml_roberta = tokenizer_xml_roberta.batch_encode_plus(
    train_text_xml_roberta,
    pad_to_max_length=True,
    max_length=max_len
)

test_encoded_xml_roberta = tokenizer_xml_roberta.batch_encode_plus(
    test_text_xml_roberta,
    pad_to_max_length=True,
    max_length=max_len
)

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(
    train_encoded_xml_roberta['input_ids'], train_data_def.label.values, 
    test_size=0.2, random_state=2020
)

x_test = test_encoded_xml_roberta['input_ids']

In [ ]:
auto = tf.data.experimental.AUTOTUNE

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(batch_size)
    .prefetch(auto)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(batch_size)
    .cache()
    .prefetch(auto)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(batch_size)
)

### ADAM, ADAMAX y SGD

**ADAM**

Es un método de optimización estocástica que solo requiere gradientes de primer orden con relativa poca memoria. 

Calcula las tasas de aprendizaje adaptativo individuales para los diferentes parámetros a partir de estimaciones del primer y segundo momento de los gradientes. El nombre proviene de ADAptative Moment estimation. 
Está pensado para combinar las ventajas de dos métodos recientemente populares: AdaGrad que funciona bien con gradientes dispersos y RMSProp que funciona bien en configuraciones on-line y no estacionarias.

Algunas de sus ventajas son:

* Las magnitudes de las actualizaciones de los parámetros son invariantes al cambio de escala del gradiente.

* Los tamaños de los steps están aproximadamente delimitados por el hiperparámetro stepsize.

* No requiere un parámetro estacionario objetivo.

* Trabaja con gradientes reducidos.

Su algoritmo sería el siguiente:

In [ ]:
response = requests.get('https://miro.medium.com/max/1100/1*zfdW5zAyQxge85gA_mFPYg.png')
image_bytes = io.BytesIO(response.content)

img = PIL.Image.open(image_bytes)
img

El principal objetivo es minimizar el valor de la función estocástica objetivo. La esperanza de dicha función es el parámetro theta. Con los diferentes f sub 0, 1, 2,..., T, denotamos las realizaciones de los subsecuentes steps.

La estocasticidad puede provenir de la evaluación en submuestras aleatorias (minibatches) de puntos de datos, o surgir del ruido inherente a la propia función.

Con gt = ∇θft(θ) denotamos el gradiente, es decir, el vector de las derivadas parciales de f con respecto a θ evaluado a timestep t.

El algoritmo actualiza las medias móviles exponenciales del gradiente (mt) y el cuadrado del gradiente (vt) donde los hiperparametros beta1 y beta2 controlan las tasas de caída exponencial de estas medias móviles, tanto gt como vt.

Las medias móviles en si mismas son estimadores del primer momento (la media) y del segundo momento (la varianza) del gradiente. Sin embargo, dichas medias móviles, son inicializadas como vectores 0, lo que lleva a que las estimaciones estén sesgadas hacia 0, especialmente durante los pasos iniciales, y también cuando los ratios de caída son pequeños (es decir, cuando las betas tienden a 1). Aunque esta inicialización es fácilmente contrarrestada en el **bias-corrected** estimando los parámetros indicados en el Compute bias-corrected first moment estimate y Compute bias-corrected second raw moment estimate.

En Adam, la regla de actualización para pesos individuales es escalar sus gradientes inversamente proporcionales a un (escalado) L^2 normal de sus gradientes individuales actuales y pasados.

Para conocer en más profundidad la **regla de actualizacion de Adam**, el **bias-corrected** o un análisis teórico de la convergencia de Adam in online convex programming, consultar "ADAM: A METHOD FOR STOCHASTIC OPTIMIZATION" de Diederik P. Kingma y Jimmy Lei Ba.

### ADAMAX

Este modelo se basa en Adam, pero generalizando la regla de actualización L^2 a una L^p. Esta variante se vuelve numéricamente inestable para grandes p, es decir cuando p->∞, pero en este caso, surge un algoritmo que estabilizaría la función (cuya demostración matemática no incluiré en este trabajo. Puede consultarse fácilmente en "ADAM: A METHOD FOR STOCHASTIC OPTIMIZATION" de Diederik P. Kingma y Jimmy Lei Ba).

A continuación, veremos el algoritmo completo de ADAMAX, una vez sustituido por la nueva regla de actualización, quedando así como una variante de Adam basada en un p que tiende a infinito.

Adamax es a veces superior a Adam, especialmente en modelos con embeddings.

In [ ]:
response = requests.get('https://paperswithcode.com/media/methods/Screen_Shot_2020-05-28_at_6.15.37_PM_apRrZCo.png')
image_bytes = io.BytesIO(response.content)

img = PIL.Image.open(image_bytes)
img

### SGD

Este optimizador, realizara una actualización de parámetros para cada ejemplo de entrenamiento y su label.

θ=θ−η⋅∇θJ(θ;x(i);y(i))

La fluctuación del SGD salta a mínimos locales nuevos y potencialmente mejores. Por otro lado, esto finalmente complica la convergencia al mínimo exacto, ya que SGD seguirá excediéndose. Sin embargo, se ha demostrado que cuando disminuimos lentamente el learning rate (la tasa de aprendizaje), SGD converge a un mínimo local o global para la optimización no convexa y convexa, respectivamente.

Como queda bien ilustrado en el siguiente grafico.

In [ ]:
response = requests.get('https://www.jeremyjordan.me/content/images/2018/02/Screen-Shot-2018-02-24-at-11.47.09-AM.png')
image_bytes = io.BytesIO(response.content)

img = PIL.Image.open(image_bytes)
img

No nos excederemos más en este optimizador, ya que no es tan potente para el problema que estudiamos como los otros dos optimizadores.

### Fine Tuning

Primero comprobaremos el **learning rate**.

Esta parte es muy importante, puesto que:

* Si la tasa de entrenamiento es demasiado pequeña, el entrenamiento, aún siendo más confiable, requiere de más steps.

* Si la tasa de entrenamiento es demasiado grande, es posible que el entrenamiento no consiga la convergencia esperada, incluso podría divergir.

Si bien en el Algorithm 1, del ADAM: A METHOD FOR STOCHASTIC OPTIMIZATION de Diederik P. Kingma y Jimmy Lei Ba, muestra que una buena configuración predeterminada para los problemas de aprendizaje automático probados es lr = 0.001, β1 = 0.9, β2 = 0.999 and E = 10−8, en este problema en concreto, parece funcionar mejor un **lr = 2e-5**.


In [ ]:
fine_tuning_lr = pd.read_csv("../input/fine-tuning/table tuning lr.csv", sep = ';')

fine_tuning_lr

Y ahora ajustaremos **epsilon**, al cual definiremos en 5e-08.

In [ ]:
fine_tuning_lr = pd.read_csv("../input/fine-tuning/epsilon table.csv", sep = ';')

fine_tuning_lr

In [ ]:
with strategy.scope():
    # Primero cargamos la capa de codificador.
    transformer_encoder = TFAutoModel.from_pretrained(model_xml_roberta)

    # Definimos los inputs tokenizados
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")

    # Ahora, codificamos los inputs segun el encoder que hemos definido anteriormente.
    sequence_output = transformer_encoder(input_ids)[0]

    # Extraemos los tokens utilizados para clasificar, en este caso <s>
    cls_token = sequence_output[:, 0, :]

    # La última capa es la que pasamos a través de softmax para la aplicación del uso de probabilidades. Con 3 niveles, ya que son 3 posibles resultados (0,1 y 2)
    out = Dense(3, activation='softmax')(cls_token)

    # Construimos y compilamos el modelo.
    model = Model(inputs=input_ids, outputs=out)
    model.compile(
        Adam(lr=2e-5,beta_1=0.9, beta_2=0.999, epsilon=5e-08), 
        loss='sparse_categorical_crossentropy', 
        metrics=['accuracy']
    )

model.summary()

In [ ]:
n_steps = len(x_train) // batch_size

train_history_xml_roberta_def = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=3
)

In [ ]:
# list all data in history
print(train_history_xml_roberta_def.history.keys())
# summarize history for loss
plt.plot(train_history_xml_roberta_def.history['loss'])
plt.plot(train_history_xml_roberta_def.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for accuracy
plt.plot(train_history_xml_roberta_def.history['accuracy'])
plt.plot(train_history_xml_roberta_def.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

Podemos comprobar que las curvas tienen un comportamiento adecuado, y obtenemos un modelo con una precisión en el conjunto de validación cercana al 90% y con una función de pérdida muy baja, lo que pronostica un buen ajuste con los datos del test.

# viii. Conclusiones finales

Se ha conseguido alcanzar tanto el objetivo general como los específicos. 

Alcanzando finalmente un modelo que pronostica la relación semántica entre las dos frases cualesquiera que le proporcionemos en cualquier idioma, con una precisión de casi un noventa por ciento.

Y también queda evidenciado que todo esto ha sido materialmente posible gracias a la implementación de unidades de procesamiento tensorial (TPUs), sin las cuales, realizar un análisis de este tipo requería una cantidad de tiempo prohibitiva, mientras que utilizando estas unidades, cada cálculo apenas ha necesitado unos pocos minutos.

# ix. Generating & Submitting Predictions

## Test en Data Science y Test en Kaggle

Llegados a este punto, se requiere aclarar la diferencia entre ambos conceptos:

* Test en Data Science: Se utiliza para evaluar la capacidad de un modelo candidato a la hora de analizar datos, extraer información, sugerir conclusiones y respaldar la toma de decisiones, y los datos surgen de un segmento de datos del conjunto de datos a analizar. Dichos datos nunca son conocidos por el modelo en su fase de entrenamiento, tan solo son introducidos al final de esta para comprobar la calidad del mismo.

* Test en Kaggle: Se trata de un conjunto de datos, sin los labels correspondientes, proporcionados por Kaggle para comprobar la puntuación obtenida en cada competición ofrecida por la plataforma.


In [ ]:
test_preds = model.predict(test_dataset, verbose=1)
submission['prediction'] = test_preds.argmax(axis=1)

In [ ]:
submission.head()

In [ ]:
submission.to_csv("submission.csv", index = False)

# x. Bibliografía

* Jeff Dean, Rajat Monga: TensorFlow. Disponible en:https://www.tensorflow.org/

* Towards Data Science Inc. TowardsDataScience. Disponible en:https://towardsdatascience.com/how-to-use-dataset-in-tensorflow-c758ef9e4428

* Diederik P. Kingma y Jimmy Lei Ba: "ADAM: A METHOD FOR STOCHASTIC OPTIMIZATION", *a conference paper at ICLR 2015*, disponible en: https://arxiv.org/pdf/1412.6980.pdf

* Tarang Shah: "About Train, Validation and Test Sets in Machine Learning", *towards data science*, 06/12/2017. Disponible en: https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7

* Alexis Conneau, Kartikay Khandelwal, Naman Goyal, Vishrav Chaudhary, Guillaume Wenzek, Francisco Guzmán, Edouard Grave, Myle Ott, Luke Zettlemoyer, Veselin Stoyanov: "Unsupervised Cross-lingual Representation Learning at Scale", *Cornell University*, 05/11/2019. Disponible en: https://arxiv.org/abs/1911.02116

* Hugging Face, Inc.https://huggingface.co/

* Fernando Pérez: Colaboratory. Disponible en: https://colab.research.google.com/

* Nick Doiron: "A whole world of BERT", *Medium*, 30/01/2020. Disponible en: https://medium.com/@mapmeld/a-whole-world-of-bert-f20d6bd47b2f

* Rick Merritt: "BERT Does Europe: AI Language Model Learns German, Swedish", *Blogs Nvidia*, 23/12/2019. Disponible en: https://blogs.nvidia.com/blog/2019/12/23/bert-ai-german-swedish/

* xhlulu: "Concise Keras XLM-R on TPU", *kaggle public notebooks*, 01/08/20. Disponible en: https://www.kaggle.com/xhlulu/contradictory-watson-concise-keras-xlm-r-on-tpu

* AdityaMishra: "NLI - Data Translation Augmentation", *kaggle public notebooks*, 01/08/20. Disponible en: https://www.kaggle.com/aditya08/nli-data-translation-augmentation

* Yih-Dar SHIEH: "More NLI datasets - Hugging Face nlp library", *kaggle public notebooks*, 01/08/20. Disponible en: https://www.kaggle.com/yihdarshieh/more-nli-datasets-hugging-face-nlp-library

* Tom Dietterich: "Overfitting and Undercomputing in Machine Learning", *Departament of Computer Science, Oregon State University, Corvallis*. Disponible en: https://dl.acm.org/doi/pdf/10.1145/212094.212114?casa_token=vqzbFaLf3noAAAAA%3AN7JvZ6jkm5st8nq0j5uodFET_r6Cc7ompQBa5y6HoiiOjp0_Q5R2k8gU_5lH5K19MKZWPQP4mNOVuw

* Sebastien C. Wong, Adam Gatt,Victor Stamatescu and Mark D. McDonnell: "Understanding data augmentation for classification: when to warp?", *2016 IEEE*. Disponible en: https://arxiv.org/pdf/1609.08764.pdf

* Alexis Conneau, Kartikay Khandelwal, Naman Goyal, Vishrav Chaudhary, Guillaume Wenzek, Francisco Guzman, Edouard Grave, Myle Ott, Luke Zettlemoyer y Veselin Stoyanov: "Unsupervised Cross-lingual Representation Learning at Scale". Disponible en: https://arxiv.org/pdf/1911.02116.pdf

* Sebastian Ruder: "An overview of gradient descent optimization algorithms", 19/01/2016. Disponible en: https://ruder.io/optimizing-gradient-descent/